# Task 1

---

## Web scraping and analysis

This Jupyter notebook includes some code to get you started with web scraping. We will use a package called `BeautifulSoup` to collect the data from the web. Once you've collected your data and saved it into a local `.csv` file you should start with your analysis.

### Scraping data from Skytrax

If you visit [https://www.airlinequality.com] you can see that there is a lot of data there. For this task, we are only interested in reviews related to British Airways and the Airline itself.

If you navigate to this link: [https://www.airlinequality.com/airline-reviews/british-airways] you will see this data. Now, we can use `Python` and `BeautifulSoup` to collect all the links to the reviews and then to collect the text data on each of the individual review links.

In [19]:
#import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
import nltk
import re
from wordcloud import WordCloud, STOPWORDS
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
nltk.download('omw-1.4')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [20]:
base_url = "https://www.airlinequality.com/airline-reviews/british-airways"
pages = 30
page_size = 100
reviews = []
recommendations = []
for i in range(1, pages + 1):
    print(f"Scraping page {i}")
    # Create URL to collect links from paginated data
    url = f"{base_url}/page/{i}/?sortby=post_date%3ADesc&pagesize={page_size}"
    # Collect HTML data from this page
    response = requests.get(url)
    # Parse content
    content = response.content
    parsed_content = BeautifulSoup(content, 'html.parser')
    # Find all review div elements and append their text content to the 'reviews' list
    for para in parsed_content.find_all("div", {"class": "text_content"}):
        reviews.append(para.get_text())
    # Find all recommendation elements and append their text content to the 'recommendations' list
    recommendation_tags = parsed_content.find_all(string=["no", "yes"])
    for rec in recommendation_tags:
        recommendations.append(rec)
    print(f"   ---> {len(reviews)} total reviews")
    print(f"   ---> {len(recommendations)} total recommendations")
# Print the number of reviews and recommendations scraped in total
print(f"Total reviews scraped: {len(reviews)}")
print(f"Total recommendations scraped: {len(recommendations)}")


Scraping page 1
   ---> 100 total reviews
   ---> 100 total recommendations
Scraping page 2
   ---> 200 total reviews
   ---> 200 total recommendations
Scraping page 3
   ---> 300 total reviews
   ---> 300 total recommendations
Scraping page 4
   ---> 400 total reviews
   ---> 400 total recommendations
Scraping page 5
   ---> 500 total reviews
   ---> 500 total recommendations
Scraping page 6
   ---> 600 total reviews
   ---> 600 total recommendations
Scraping page 7
   ---> 700 total reviews
   ---> 700 total recommendations
Scraping page 8
   ---> 800 total reviews
   ---> 800 total recommendations
Scraping page 9
   ---> 900 total reviews
   ---> 900 total recommendations
Scraping page 10
   ---> 1000 total reviews
   ---> 1000 total recommendations
Scraping page 11
   ---> 1100 total reviews
   ---> 1100 total recommendations
Scraping page 12
   ---> 1200 total reviews
   ---> 1200 total recommendations
Scraping page 13
   ---> 1300 total reviews
   ---> 1300 total recommendations


In [21]:
df = pd.DataFrame()
df["reviews"] = reviews
df["recommendations"] = recommendations
df.head(50)

,reviews,recommendations
0,✅ Trip Verified | I am a British Airways Gold ...,no
1,Not Verified | Another case of reviewing Brit...,no
2,✅ Trip Verified | On the memorable day of Janu...,yes
3,Not Verified | I did not actually get to fly w...,no
4,✅ Trip Verified | We had possibly the worse ch...,no
5,✅ Trip Verified | I flew to LHR from ATH in C...,yes
6,✅ Trip Verified | I like the British Airways ...,yes
7,✅ Trip Verified | I have come to boarding and...,yes
8,✅ Trip Verified | Stinking nappies being chang...,no
9,✅ Trip Verified | Worst service ever. Lost bag...,no


In [22]:
import os

# Create the "data" directory if it doesn't exist
data_dir = 'data'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)

# Save the DataFrame to a CSV file
df.to_csv(os.path.join(data_dir, 'BA_reviews.csv'), index=False)


In [23]:
df.to_csv("data/BA_reviews.csv")

In [24]:
df = pd.read_csv("data/BA_reviews.csv")
df

,Unnamed: 0,reviews,recommendations
0,0,✅ Trip Verified | I am a British Airways Gold ...,no
1,1,Not Verified | Another case of reviewing Brit...,no
2,2,✅ Trip Verified | On the memorable day of Janu...,yes
3,3,Not Verified | I did not actually get to fly w...,no
4,4,✅ Trip Verified | We had possibly the worse ch...,no
...,...,...,...
2995,2995,A380 LHR-IAD. After a visit to the Concorde Ro...,yes
2996,2996,Club World: Just flown London Gatwick to Las V...,no
2997,2997,We travelled on Flight BA11 from Heathrow to S...,no
2998,2998,Punta Cana - Gatwick. World Traveller Plus cab...,no


Congratulations! Now you have your dataset for this task! The loops above collected 3000 reviews by iterating through the paginated pages on the website. However, if you want to collect more data, try increasing the number of pages!

 The next thing that you should do is clean this data to remove any unnecessary text from each of the rows. For example, "✅ Trip Verified" can be removed from each row if it exists, as it's not relevant to what we want to investigate.

<h1><b><u>Data Cleaning</u></b></h1>

### checking for content in Reviews column - use [ ] i.e df['column name][index]

In [31]:
df['reviews'][0]

'✅ Trip Verified | I am a British Airways Gold Guest List member (with Lifetime privileges). In spite of that I have given up Britsh wherever possible. I take ten or more trans ocean trips a year. I am fortunate to be able to fly First Class - unfortunately British no longer has a First Class product - the sell one by that name at First Class prices but in fairness deliver what you would consider a very good Business Class experience. I should point out that there are airlines who sell Business Class that in fact are better than any British First Class experience (Qatar, Emirates). I have flown many millions of miles on British. Last week I completed an around the world trip on Emirates and Qantas all A380. In the millions of BA miles I never had an experience as good as what I experience on EK and Qantas. A very long time ago the food and beverage service on BA was truly extraordinary - today it would be a bit of a stretch to even call it ordinary - so sad a decline. Seat comfort (in 

In [36]:
df.reviews= df.reviews.str.split('|',expand=True)
df.head()


,Unnamed: 0,reviews,recommendations
0,0,I am a British Airways Gold Guest List member...,no
1,1,Another case of reviewing British Airways fl...,no
2,2,"On the memorable day of January 9th, my famil...",yes
3,3,I did not actually get to fly with BA as they...,no
4,4,We had possibly the worse check-in experience...,no


This code splits the reviews column of a dataframe df by the pipe (|) character, and expands the result into a new column. It then displays the first five rows of the dataframe using the head() method.

